# Quy trình Human-in-the-Loop với Microsoft Agent Framework

## 🎯 Mục tiêu học tập

Trong notebook này, bạn sẽ học cách triển khai quy trình **human-in-the-loop** bằng cách sử dụng `RequestInfoExecutor` của Microsoft Agent Framework. Mẫu thiết kế mạnh mẽ này cho phép bạn tạm dừng quy trình AI để thu thập ý kiến từ con người, giúp các agent trở nên tương tác và trao quyền kiểm soát các quyết định quan trọng cho con người.

## 🔄 Human-in-the-Loop là gì?

**Human-in-the-loop (HITL)** là một mẫu thiết kế trong đó các agent AI tạm dừng thực thi để yêu cầu ý kiến từ con người trước khi tiếp tục. Điều này rất cần thiết cho:

- ✅ **Quyết định quan trọng** - Nhận sự phê duyệt từ con người trước khi thực hiện các hành động quan trọng
- ✅ **Tình huống mơ hồ** - Để con người làm rõ khi AI không chắc chắn
- ✅ **Sở thích của người dùng** - Hỏi người dùng chọn giữa nhiều tùy chọn
- ✅ **Tuân thủ & an toàn** - Đảm bảo sự giám sát của con người đối với các hoạt động được quy định
- ✅ **Trải nghiệm tương tác** - Xây dựng các agent hội thoại phản hồi theo ý kiến người dùng

## 🏗️ Cách hoạt động trong Microsoft Agent Framework

Framework cung cấp ba thành phần chính cho HITL:

1. **`RequestInfoExecutor`** - Một executor đặc biệt tạm dừng quy trình và phát ra một `RequestInfoEvent`
2. **`RequestInfoMessage`** - Lớp cơ sở cho các payload yêu cầu được gửi đến con người
3. **`RequestResponse`** - Liên kết phản hồi của con người với các yêu cầu ban đầu bằng cách sử dụng `request_id`

**Mẫu quy trình:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Ví dụ của chúng ta: Đặt phòng khách sạn với xác nhận từ người dùng

Chúng ta sẽ xây dựng dựa trên quy trình điều kiện bằng cách thêm xác nhận từ con người **trước khi** đề xuất các điểm đến thay thế:

1. Người dùng yêu cầu một điểm đến (ví dụ: "Paris")
2. `availability_agent` kiểm tra xem có phòng trống không
3. **Nếu không có phòng** → `confirmation_agent` hỏi "Bạn có muốn xem các điểm đến thay thế không?"
4. Quy trình **tạm dừng** bằng cách sử dụng `RequestInfoExecutor`
5. **Con người phản hồi** "có" hoặc "không" qua đầu vào console
6. `decision_manager` định tuyến dựa trên phản hồi:
   - **Có** → Hiển thị các điểm đến thay thế
   - **Không** → Hủy yêu cầu đặt phòng
7. Hiển thị kết quả cuối cùng

Điều này minh họa cách trao quyền kiểm soát cho người dùng đối với các đề xuất của agent!

---

Hãy bắt đầu nào! 🚀


## Bước 1: Nhập các thư viện cần thiết

Chúng ta nhập các thành phần tiêu chuẩn của Agent Framework cùng với **các lớp cụ thể liên quan đến con người trong quy trình**:
- `RequestInfoExecutor` - Bộ thực thi tạm dừng quy trình để nhận đầu vào từ con người
- `RequestInfoEvent` - Sự kiện được phát ra khi yêu cầu đầu vào từ con người
- `RequestInfoMessage` - Lớp cơ bản cho các tải trọng yêu cầu được định kiểu
- `RequestResponse` - Liên kết phản hồi của con người với các yêu cầu
- `WorkflowOutputEvent` - Sự kiện để phát hiện đầu ra của quy trình


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Bước 2: Định nghĩa các Mô hình Pydantic cho Kết quả Có cấu trúc

Các mô hình này định nghĩa **schema** mà các agent sẽ trả về. Chúng tôi giữ tất cả các mô hình từ quy trình làm việc có điều kiện và thêm:

**Mới cho Human-in-the-Loop:**
- `HumanFeedbackRequest` - Lớp con của `RequestInfoMessage` định nghĩa payload yêu cầu gửi đến con người
  - Bao gồm `prompt` (câu hỏi cần hỏi) và `destination` (ngữ cảnh về thành phố không khả dụng)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Bước 3: Tạo Công Cụ Đặt Phòng Khách Sạn

Công cụ giống như trong quy trình làm việc có điều kiện - kiểm tra xem phòng có sẵn tại điểm đến hay không.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Bước 4: Định nghĩa các hàm điều kiện cho định tuyến

Chúng ta cần **bốn hàm điều kiện** cho quy trình làm việc có sự can thiệp của con người:

**Từ quy trình làm việc có điều kiện:**
1. `has_availability_condition` - Định tuyến khi khách sạn CÓ sẵn
2. `no_availability_condition` - Định tuyến khi khách sạn KHÔNG sẵn

**Mới cho quy trình làm việc có sự can thiệp của con người:**
3. `user_wants_alternatives_condition` - Định tuyến khi người dùng nói "có" với các lựa chọn thay thế
4. `user_declines_alternatives_condition` - Định tuyến khi người dùng nói "không" với các lựa chọn thay thế


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Bước 5: Tạo Decision Manager Executor

Đây là **trọng tâm của mô hình con người trong vòng lặp**! `DecisionManager` là một `Executor` tùy chỉnh có chức năng:

1. **Nhận phản hồi từ con người** thông qua các đối tượng `RequestResponse`
2. **Xử lý quyết định của người dùng** (có/không)
3. **Điều hướng luồng công việc** bằng cách gửi thông điệp đến các tác nhân phù hợp

Các tính năng chính:
- Sử dụng trình trang trí `@handler` để hiển thị các phương thức như các bước trong luồng công việc
- Nhận `RequestResponse[HumanFeedbackRequest, str]` chứa cả yêu cầu ban đầu và câu trả lời của người dùng
- Tạo ra các thông điệp đơn giản "có" hoặc "không" để kích hoạt các hàm điều kiện của chúng ta


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Bước 6: Tạo Bộ Thực Thi Hiển Thị Tùy Chỉnh

Bộ thực thi hiển thị giống như trong quy trình làm việc có điều kiện - cung cấp kết quả cuối cùng dưới dạng đầu ra của quy trình làm việc.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Bước 7: Tải các biến môi trường

Cấu hình client LLM (GitHub Models, Azure OpenAI, hoặc OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Bước 8: Tạo các tác nhân AI và bộ thực thi

Chúng ta tạo **sáu thành phần quy trình làm việc**:

**Các tác nhân (được bao bọc trong AgentExecutor):**
1. **availability_agent** - Kiểm tra tình trạng phòng khách sạn bằng công cụ
2. **confirmation_agent** - 🆕 Chuẩn bị yêu cầu xác nhận từ con người
3. **alternative_agent** - Đề xuất các thành phố thay thế (khi người dùng đồng ý)
4. **booking_agent** - Khuyến khích đặt phòng (khi có phòng trống)
5. **cancellation_agent** - 🆕 Xử lý thông báo hủy (khi người dùng từ chối)

**Các bộ thực thi đặc biệt:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` tạm dừng quy trình làm việc để nhận thông tin từ con người
7. **decision_manager** - 🆕 Bộ thực thi tùy chỉnh định tuyến dựa trên phản hồi của con người (đã được định nghĩa ở trên)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Bước 9: Xây dựng quy trình làm việc với sự tham gia của con người

Bây giờ chúng ta sẽ xây dựng biểu đồ quy trình làm việc với **định tuyến có điều kiện**, bao gồm cả đường dẫn có sự tham gia của con người:

**Cấu trúc quy trình làm việc:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Các cạnh chính:**
- `availability_agent → confirmation_agent` (khi không có phòng)
- `confirmation_agent → prepare_human_request` (chuyển đổi loại)
- `prepare_human_request → request_info_executor` (tạm dừng để con người tham gia)
- `request_info_executor → decision_manager` (luôn luôn - cung cấp RequestResponse)
- `decision_manager → alternative_agent` (khi người dùng nói "có")
- `decision_manager → cancellation_agent` (khi người dùng nói "không")
- `availability_agent → booking_agent` (khi có phòng)
- Tất cả các đường dẫn kết thúc tại `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Bước 10: Chạy Test Case 1 - Thành phố KHÔNG có phòng trống (Paris với Xác nhận từ Con người)

Test này minh họa **chu trình đầy đủ có sự tham gia của con người**:

1. Yêu cầu khách sạn ở Paris
2. availability_agent kiểm tra → Không có phòng
3. confirmation_agent tạo câu hỏi hướng đến con người
4. request_info_executor **tạm dừng quy trình** và phát ra `RequestInfoEvent`
5. **Ứng dụng phát hiện sự kiện và nhắc người dùng trên console**
6. Người dùng nhập "yes" hoặc "no"
7. Ứng dụng gửi phản hồi qua `send_responses_streaming()`
8. decision_manager định tuyến dựa trên phản hồi
9. Kết quả cuối cùng được hiển thị

**Mẫu chính:**
- Sử dụng `workflow.run_stream()` cho lần lặp đầu tiên
- Sử dụng `workflow.send_responses_streaming(pending_responses)` cho các lần lặp tiếp theo
- Lắng nghe `RequestInfoEvent` để phát hiện khi cần đầu vào từ con người
- Lắng nghe `WorkflowOutputEvent` để thu thập kết quả cuối cùng


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Bước 11: Chạy Test Case 2 - Thành phố CÓ Phòng (Stockholm - Không Cần Tương Tác Người Dùng)

Test này minh họa **đường dẫn trực tiếp** khi có phòng trống:

1. Yêu cầu khách sạn ở Stockholm  
2. availability_agent kiểm tra → Có phòng trống ✅  
3. booking_agent đề xuất đặt phòng  
4. display_result hiển thị xác nhận  
5. **Không cần tương tác người dùng!**

Quy trình này hoàn toàn bỏ qua đường dẫn có sự tham gia của con người khi có phòng trống.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Những Điểm Chính và Thực Hành Tốt Nhất về Human-in-the-Loop

### ✅ Những Điều Bạn Đã Học:

#### 1. **Mẫu RequestInfoExecutor**
Mẫu human-in-the-loop trong Microsoft Agent Framework sử dụng ba thành phần chính:
- `RequestInfoExecutor` - Tạm dừng luồng công việc và phát ra sự kiện
- `RequestInfoMessage` - Lớp cơ sở cho các payload yêu cầu có kiểu (hãy kế thừa lớp này!)
- `RequestResponse` - Liên kết phản hồi của con người với yêu cầu ban đầu

**Hiểu Biết Quan Trọng:**
- `RequestInfoExecutor` KHÔNG tự thu thập đầu vào - nó chỉ tạm dừng luồng công việc
- Mã ứng dụng của bạn phải lắng nghe `RequestInfoEvent` và thu thập đầu vào
- Bạn phải gọi `send_responses_streaming()` với một dict ánh xạ `request_id` tới câu trả lời của người dùng

#### 2. **Mẫu Thực Thi Streaming**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Kiến Trúc Dựa Trên Sự Kiện**
Lắng nghe các sự kiện cụ thể để kiểm soát luồng công việc:
- `RequestInfoEvent` - Cần đầu vào từ con người (luồng công việc bị tạm dừng)
- `WorkflowOutputEvent` - Kết quả cuối cùng đã sẵn sàng (luồng công việc hoàn tất)
- `WorkflowStatusEvent` - Thay đổi trạng thái (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, v.v.)

#### 4. **Executors Tùy Chỉnh với @handler**
`DecisionManager` minh họa cách tạo executors mà:
- Sử dụng decorator `@handler` để hiển thị các phương thức như các bước luồng công việc
- Nhận các thông điệp có kiểu (ví dụ: `RequestResponse[HumanFeedbackRequest, str]`)
- Điều hướng luồng công việc bằng cách gửi thông điệp tới các executors khác
- Truy cập ngữ cảnh thông qua `WorkflowContext`

#### 5. **Điều Hướng Có Điều Kiện với Quyết Định của Con Người**
Bạn có thể tạo các hàm điều kiện để đánh giá phản hồi của con người:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Ứng Dụng Thực Tế:

1. **Luồng Công Việc Phê Duyệt**
   - Nhận phê duyệt từ quản lý trước khi xử lý báo cáo chi phí
   - Yêu cầu con người xem xét trước khi gửi email tự động
   - Xác nhận các giao dịch có giá trị cao trước khi thực hiện

2. **Kiểm Duyệt Nội Dung**
   - Đánh dấu nội dung đáng ngờ để con người xem xét
   - Yêu cầu người kiểm duyệt đưa ra quyết định cuối cùng trong các trường hợp khó
   - Chuyển lên con người khi độ tin cậy của AI thấp

3. **Dịch Vụ Khách Hàng**
   - Để AI xử lý các câu hỏi thường gặp một cách tự động
   - Chuyển các vấn đề phức tạp cho nhân viên hỗ trợ
   - Hỏi khách hàng nếu họ muốn nói chuyện với con người

4. **Xử Lý Dữ Liệu**
   - Yêu cầu con người giải quyết các mục dữ liệu không rõ ràng
   - Xác nhận cách hiểu của AI về các tài liệu không rõ ràng
   - Để người dùng chọn giữa các cách hiểu hợp lệ khác nhau

5. **Hệ Thống Quan Trọng về An Toàn**
   - Yêu cầu xác nhận từ con người trước khi thực hiện các hành động không thể đảo ngược
   - Nhận phê duyệt trước khi truy cập dữ liệu nhạy cảm
   - Xác nhận các quyết định trong các ngành được quy định (y tế, tài chính)

6. **Tác Nhân Tương Tác**
   - Xây dựng bot hội thoại hỏi các câu hỏi tiếp theo
   - Tạo các trình hướng dẫn dẫn dắt người dùng qua các quy trình phức tạp
   - Thiết kế các tác nhân hợp tác với con người từng bước

### 🔄 So Sánh: Có và Không Có Human-in-the-Loop

| Tính Năng | Luồng Công Việc Có Điều Kiện | Luồng Công Việc Human-in-the-Loop |
|-----------|-----------------------------|-----------------------------------|
| **Thực Thi** | Một lần `workflow.run()` | Vòng lặp với `run_stream()` + `send_responses_streaming()` |
| **Đầu Vào Người Dùng** | Không có (hoàn toàn tự động) | Nhắc nhở tương tác qua `input()` hoặc giao diện người dùng |
| **Thành Phần** | Agents + Executors | + RequestInfoExecutor + DecisionManager |
| **Sự Kiện** | Chỉ AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, v.v. |
| **Tạm Dừng** | Không tạm dừng | Luồng công việc tạm dừng tại RequestInfoExecutor |
| **Kiểm Soát của Con Người** | Không có kiểm soát của con người | Con người đưa ra các quyết định quan trọng |
| **Trường Hợp Sử Dụng** | Tự động hóa | Hợp tác & giám sát |

### 🚀 Mẫu Nâng Cao:

#### Nhiều Điểm Quyết Định của Con Người
Bạn có thể có nhiều nút `RequestInfoExecutor` trong cùng một luồng công việc:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Xử Lý Hết Thời Gian
Triển khai hết thời gian cho phản hồi của con người:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Tích Hợp Giao Diện Người Dùng Phong Phú
Thay vì `input()`, tích hợp với giao diện web, Slack, Teams, v.v.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Human-in-the-Loop Có Điều Kiện
Chỉ yêu cầu đầu vào từ con người trong các tình huống cụ thể:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Thực Hành Tốt Nhất:

1. **Luôn Kế Thừa RequestInfoMessage**
   - Cung cấp an toàn kiểu và xác thực
   - Cho phép ngữ cảnh phong phú để hiển thị giao diện người dùng
   - Làm rõ ý định của từng loại yêu cầu

2. **Sử Dụng Lời Nhắc Mô Tả**
   - Bao gồm ngữ cảnh về điều bạn đang hỏi
   - Giải thích hậu quả của từng lựa chọn
   - Giữ câu hỏi đơn giản và rõ ràng

3. **Xử Lý Đầu Vào Không Mong Muốn**
   - Xác thực phản hồi của người dùng
   - Cung cấp giá trị mặc định cho đầu vào không hợp lệ
   - Đưa ra thông báo lỗi rõ ràng

4. **Theo Dõi Request IDs**
   - Sử dụng sự liên kết giữa request_id và phản hồi
   - Không cố gắng quản lý trạng thái một cách thủ công

5. **Thiết Kế Không Chặn**
   - Không chặn các luồng chờ đầu vào
   - Sử dụng các mẫu async xuyên suốt
   - Hỗ trợ các phiên bản luồng công việc đồng thời

### 📚 Các Khái Niệm Liên Quan:

- **Agent Middleware** - Chặn các cuộc gọi của agent (notebook trước)
- **Quản Lý Trạng Thái Luồng Công Việc** - Duy trì trạng thái luồng công việc giữa các lần chạy
- **Hợp Tác Đa Tác Nhân** - Kết hợp human-in-the-loop với các nhóm tác nhân
- **Kiến Trúc Dựa Trên Sự Kiện** - Xây dựng hệ thống phản ứng với sự kiện

---

### 🎓 Chúc Mừng!

Bạn đã nắm vững các luồng công việc human-in-the-loop với Microsoft Agent Framework! Giờ đây bạn đã biết cách:
- ✅ Tạm dừng luồng công việc để thu thập đầu vào từ con người
- ✅ Sử dụng RequestInfoExecutor và RequestInfoMessage
- ✅ Xử lý thực thi streaming với các sự kiện
- ✅ Tạo executors tùy chỉnh với @handler
- ✅ Điều hướng luồng công việc dựa trên quyết định của con người
- ✅ Xây dựng các tác nhân AI tương tác hợp tác với con người

**Đây là một mẫu quan trọng để xây dựng các hệ thống AI đáng tin cậy và có thể kiểm soát!** 🚀



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp của con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
